<a href="https://colab.research.google.com/github/UmardrazArshad/Quarter_3/blob/main/04_hello_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

## **Run Google Gemini with OPENAI-Agent SDK**

In [3]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

In [4]:
from agents import Agent, Runner, OpenAIChatCompletionsModel, enable_verbose_stdout_logging,AsyncOpenAI
from agents.run import RunConfig

# set external_llm(Gemini)
external_client = AsyncOpenAI(
    api_key = GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

# model
model = OpenAIChatCompletionsModel(
    model="gemini-1.5-flash",
    openai_client=external_client, #Assignt openai_client with any external llm (Gemini)
)

#Configuration (optional)
config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)

## **Hello world code | method one**

In [5]:
# Agent
agent : Agent = Agent(
    name = "Assistant",
    instructions= "You are a helpfull Assistant",
    model=model,
)

# Runner
# Async: Non-blocking, allows other tasks to run
result = await Runner.run(agent, "Hi")  # Waits for result but frees up thread for other tasks
print (result.final_output)

Hi there! How can I help you today?



## **Hello world code | method two**

In [6]:
import asyncio

async def main():
    # Agent
    agent : Agent = Agent(
        name = "Assistant",
        instructions= "You are a helpfull Assistant",
        model=model,
    )

    # Runner
    # Sync: Blocking, waits for completion
    result = Runner.run_sync(agent, "Hi")  # Halts execution until result is returnedprint (Result.final_output)
    print (result.final_output)
asyncio.run(main())

Hi there! How can I help you today?



## **Agent Level Custom model configuration**

In [9]:
from agents import Agent, Runner, function_tool, set_tracing_disabled
BASE_URL ="https://generativelanguage.googleapis.com/v1beta/openai/"
API_KEY =userdata.get("GEMINI_API_KEY")
MODEL_NAME ="gemini-2.0-flash"

client = AsyncOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY
)

set_tracing_disabled(disabled=True)

async def main():
    # This agent will use the custom LLM provider
    agent = Agent(
        name="Assistant",
        instructions="You only respond in haikus.",
        model=OpenAIChatCompletionsModel(model=MODEL_NAME, openai_client=client),
    )

    result = await Runner.run(agent, "Who is the founder of Pakistan?")
    print(result.final_output)


asyncio.run(main())

Jinnah, the leader,
Founded Pakistan with dreams,
A nation was born.



## **Set Model(LLM) configration on Global level**

In [13]:
from agents import Agent, Runner, function_tool, set_default_openai_api, set_default_openai_client, set_tracing_disabled

BASE_URL ="https://generativelanguage.googleapis.com/v1beta/openai/"
API_KEY =userdata.get("GEMINI_API_KEY")
MODEL_NAME ="gemini-2.0-flash"

client = AsyncOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY
)

set_default_openai_client(client=client, use_for_tracing=False)
set_default_openai_api("chat_completions")
set_tracing_disabled(disabled=True)

@function_tool
def Get_Weather(city: str) -> str:
    return f"The weather in {city} is sunny"

async def main():
    # This agent will use the custom LLM provider
    agent = Agent(
        name="Assistant",
        instructions="You only respond in haikus.",
        model=MODEL_NAME,
        tools=[Get_Weather],
    )

    result = await Runner.run(agent, "Whats the weather in kasur")
    print(result.final_output)


asyncio.run(main())

The sun shines brightly
In Kasur, a lovely day
Enjoy the weather

